In [ ]:
# imports
import numpy as np
import tensorly as tl
from timpute.decomposition import Decomposition
from timpute.tracker import tracker
from timpute.common import *
from timpute.plot import *

from timpute.test.simulated_tensors import createTensor, createCube
from timpute.direct_opt import perform_CP_DO
from timpute.cmtf import perform_CP
from tensorly.decomposition import parafac

In [ ]:
np.random.seed(10)

# tensor options
# 1 = gamma dist, 2 = normal dist, 3 = atyeo, 4 = zohar, 5 = alter
option = 1
r = 30
shape = (20,40,10)
scale = 5
miss = False
miss_perc = 0.2

# imputation options
reps = 5
impute_r = 6
maxiter = 100
drop_perc = 0.2

# graphing options
ax, f = getSetup((16,7), (3,3))

In [ ]:
if option == 1:
    if miss: tensor = createTensor(drop_perc=drop_perc, size=shape, rank=r, distribution="gamma", scale=scale, par=2)
    else: tensor = createTensor(size=shape, rank=r, distribution="gamma", scale=scale, par=2)
elif option == 2:
    if miss: tensor = createTensor(drop_perc=drop_perc, size=shape, rank=r, distribution="uniform", scale=scale)
    else: tensor = createTensor(size=shape, rank=r, distribution="uniform", scale=scale)
elif option == 3:
    from tensordata.atyeo import data as atyeo
    tensor = atyeo()
elif option == 4:
    from tensordata.zohar import data as zohar
    tensor = zohar()
elif option == 5:
    from tensordata.alter import data as alter
    tensor = alter()['Fc']
    matrix = alter()['gp120']

drop_num = int(np.sum(np.isfinite(tensor))*drop_perc)

## Start of methods

In [ ]:
dopt_tracker = tracker(tensor,track_runtime=True)
dopt_method = Decomposition(tensor,method=perform_CP_DO, max_rr=impute_r)
dopt_method.Q2X_entry(drop=drop_num, repeat=reps, maxiter=maxiter, comparePCA=False, callback=dopt_tracker)
dopt_tracker.combine()

In [ ]:
als_tracker = tracker(tensor, track_runtime=True)
# als_method = Decomposition(tensor,method=parafac, max_rr=impute_r)
# als_method.Q2X_entry(drop=drop_num, repeat=reps, maxiter=maxiter, comparePCA=False, callback=als_tracker)
# als_tracker.combine()

In [ ]:
cls_tracker = tracker(tensor, track_runtime=True)
cls_method = Decomposition(tensor,method=perform_CP, max_rr=impute_r)
cls_method.Q2X_entry(drop=drop_num, repeat=reps, maxiter=maxiter, comparePCA=False, callback=cls_tracker)
cls_tracker.combine()

In [ ]:
q2xentry(ax[0], dopt_method, methodname = "Direct Opt CP")
# q2xentry(ax[0], als_method, methodname = "Tensorpack CP")
q2xentry(ax[0], cls_method, methodname = "Censored Least Squared CP")

dopt_tracker.plot_iteration(ax[1], methodname="Direct Opt CP")
# als_tracker.plot_iteration(ax[1], methodname="Alternating Least Squares CP")
cls_tracker.plot_iteration(ax[1], methodname="Censored Least Squared CP")

dopt_tracker.plot_runtime(ax[3], methodname="Direct Opt CP")
# als_tracker.plot_runtime(ax[4], methodname="Alternating Least Squares CP")
cls_tracker.plot_runtime(ax[5], methodname="Censored Least Squared CP")

dopt_tracker.plot_iteration(ax[6], methodname="Direct Opt CP", average=False)
# als_tracker.plot_iteration(ax[7], methodname="Alternating Least Squared CP", average=False)
cls_tracker.plot_iteration(ax[8], methodname="Censored Least Squared CP", average=False)



print(dopt_method.entryQ2X)
print(cls_method.entryQ2X)
f

## Test Functions

In [ ]:
als_tracker.begin()
parafac(tensor, 6, n_iter_max=100, callback=als_tracker)